### Reference Baseline
Random Forest baseline:
- Test RMSE ≈ 48.8k
- CV RMSE ≈ 49.1k ± 0.65k

---

## Preprocessing

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import root_mean_squared_error, r2_score

In [2]:
path = '../data/raw/'
df = pd.read_csv(os.path.join(path, 'housing.csv'))
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [3]:
X = df.drop("median_house_value", axis=1)
y = df["median_house_value"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [4]:
X.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
ocean_proximity         0
dtype: int64

In [5]:
imputer = SimpleImputer(strategy="median")

X_train["total_bedrooms"] = imputer.fit_transform(
    X_train[["total_bedrooms"]]
)

X_test["total_bedrooms"] = imputer.transform(
    X_test[["total_bedrooms"]]
)

In [6]:
print("Original ocean_proximity unique values:")
print(X_train["ocean_proximity"].value_counts())

Original ocean_proximity unique values:
ocean_proximity
<1H OCEAN     7341
INLAND        5227
NEAR OCEAN    2086
NEAR BAY      1854
ISLAND           4
Name: count, dtype: int64


In [7]:
# Create encoder
encoder = OneHotEncoder(
    sparse_output=False,      # Get dense array (not sparse matrix)
    handle_unknown='ignore'   # Ignore new categories in test
)

# Fit on train, transform train
encoded_train = encoder.fit_transform(X_train[["ocean_proximity"]])

# Transform test (uses train categories only)
encoded_test = encoder.transform(X_test[["ocean_proximity"]])

# Get proper column names
encoded_cols = encoder.get_feature_names_out()

print("Encoded column names:", encoded_cols)
print("Train shape:", encoded_train.shape)  # (16512, 5)
print("Test shape:", encoded_test.shape)    # (4128, 5)

Encoded column names: ['ocean_proximity_<1H OCEAN' 'ocean_proximity_INLAND'
 'ocean_proximity_ISLAND' 'ocean_proximity_NEAR BAY'
 'ocean_proximity_NEAR OCEAN']
Train shape: (16512, 5)
Test shape: (4128, 5)


In [8]:
# Convert to DataFrames with correct indices
encoded_train_df = pd.DataFrame(
    encoded_train, columns=encoded_cols, index=X_train.index
)
encoded_test_df = pd.DataFrame(
    encoded_test, columns=encoded_cols, index=X_test.index
)

print("\nFirst few rows:")
encoded_train_df.head()


First few rows:


,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
14196,0.0,0.0,0.0,0.0,1.0
8267,0.0,0.0,0.0,0.0,1.0
17445,0.0,0.0,0.0,0.0,1.0
14265,0.0,0.0,0.0,0.0,1.0
2271,0.0,1.0,0.0,0.0,0.0


In [9]:
# Drop original ocean_proximity and add encoded columns
X_train = X_train.drop("ocean_proximity", axis=1).join(encoded_train_df)
X_test = X_test.drop("ocean_proximity", axis=1).join(encoded_test_df)

print("Final shapes:")
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)

print(encoded_train)

Final shapes:
X_train: (16512, 13)
X_test: (4128, 13)
[[0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 ...
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]]


In [10]:
X_train.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
14196,-117.03,32.71,33.0,3126.0,627.0,2300.0,623.0,3.2596,0.0,0.0,0.0,0.0,1.0
8267,-118.16,33.77,49.0,3382.0,787.0,1314.0,756.0,3.8125,0.0,0.0,0.0,0.0,1.0
17445,-120.48,34.66,4.0,1897.0,331.0,915.0,336.0,4.1563,0.0,0.0,0.0,0.0,1.0


---

## Feature Engineering

In [11]:
import numpy as np

def add_engineered_features(df):
    df = df.copy()
    
    # Avoid division by zero
    df["rooms_per_household"] = df["total_rooms"] / df["households"]
    df["bedrooms_per_room"] = df["total_bedrooms"] / df["total_rooms"]
    df["population_per_household"] = df["population"] / df["households"]
    
    return df


In [12]:
X_train_fe = add_engineered_features(X_train)
X_test_fe = add_engineered_features(X_test)

In [13]:
X_train_fe.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN,rooms_per_household,bedrooms_per_room,population_per_household
14196,-117.03,32.71,33.0,3126.0,627.0,2300.0,623.0,3.2596,0.0,0.0,0.0,0.0,1.0,5.017657,0.200576,3.691814
8267,-118.16,33.77,49.0,3382.0,787.0,1314.0,756.0,3.8125,0.0,0.0,0.0,0.0,1.0,4.473545,0.232703,1.738095
17445,-120.48,34.66,4.0,1897.0,331.0,915.0,336.0,4.1563,0.0,0.0,0.0,0.0,1.0,5.645833,0.174486,2.723214


In [14]:
X_train_fe[[
    "rooms_per_household",
    "bedrooms_per_room",
    "population_per_household"
]].describe()

,rooms_per_household,bedrooms_per_room,population_per_household
count,16512.000000,16512.000000,16512.000000
mean,5.435235,0.212858,3.096961
std,2.387375,0.057995,11.578744
min,0.888889,0.100000,0.692308
25%,4.452055,0.175178,2.428799
50%,5.235874,0.202808,2.817240
75%,6.061037,0.239501,3.280000
max,141.909091,1.000000,1243.333333


---

## Training Random Forest Regressor for New Engineered Features

In [15]:
from sklearn.ensemble import RandomForestRegressor
rt = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)
rt.fit(X_train_fe, y_train)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",200
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsample

In [16]:
y_train_pred = rt.predict(X_train_fe)
y_test_pred = rt.predict(X_test_fe)

train_rmse = root_mean_squared_error(y_train, y_train_pred)
test_rmse = root_mean_squared_error(y_test, y_test_pred)

print(f"Train RMSE: {train_rmse:.2f}")
print(f"Test RMSE: {test_rmse:.2f}")

train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"Train R²: {train_r2:.2f}")
print(f"Test R²: {test_r2:.2f}")

Train RMSE: 18294.11
Test RMSE: 50064.79
Train R²: 0.97
Test R²: 0.81


---

## Use Gradient Boosting Regressor

In [17]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

hgb_model = HistGradientBoostingRegressor(
    max_depth=8,
    learning_rate=0.1,
    max_iter=200,
    random_state=42
)

hgb_model.fit(X_train_fe, y_train)

,"loss loss: {'squared_error', 'absolute_error', 'gamma', 'poisson', 'quantile'}, default='squared_error'The loss function to use in the boosting process. Note that the""squared error"", ""gamma"" and ""poisson"" losses actually implement""half least squares loss"", ""half gamma deviance"" and ""half poissondeviance"" to simplify the computation of the gradient. Furthermore,""gamma"" and ""poisson"" losses internally use a log-link, ""gamma""requires ``y > 0`` and ""poisson"" requires ``y >= 0``.""quantile"" uses the pinball loss... versionchanged:: 0.23 Added option 'poisson'... versionchanged:: 1.1 Added option 'quantile'... versionchanged:: 1.3 Added option 'gamma'.",'squared_error'
,"quantile quantile: float, default=NoneIf loss is ""quantile"", this parameter specifies which quantile to be estimatedand must be between 0 and 1.",None
,"learning_rate learning_rate: float, default=0.1The learning rate, also known as *shrinkage*. This is used as amultiplicative factor for the leaves values. Use ``1`` for noshrinkage.",0.1
,"max_iter max_iter: int, default=100The maximum number of iterations of the boosting process, i.e. themaximum number of trees.",200
,"max_leaf_nodes max_leaf_nodes: int or None, default=31The maximum number of leaves for each tree. Must be strictly greaterthan 1. If None, there is no maximum limit.",31
,"max_depth max_depth: int or None, default=NoneThe maximum depth of each tree. The depth of a tree is the number ofedges to go from the root to the deepest leaf.Depth isn't constrained by default.",8
,"min_samples_leaf min_samples_leaf: int, default=20The minimum number of samples per leaf. For small datasets with lessthan a few hundred samples, it is recommended to lower this valuesince only very shallow trees would be built.",20
,"l2_regularization l2_regularization: float, default=0The L2 regularization parameter penalizing leaves with small hessians.Use ``0`` for no regularization (default).",0.0
,"max_features max_features: float, default=1.0Proportion of randomly chosen features in each and every node split.This is a form of regularization, smaller values make the trees weakerlearners and might prevent overfitting.If interaction constraints from `interaction_cst` are present, only allowedfeatures are taken into account for the subsampling... versionadded:: 1.4",1.0
,"max_bins max_bins: int, default=255The maximum number of bins to use for non-missing values. Beforetraining, each feature of the input array `X` is binned intointeger-valued bins, which allows for a much faster training stage.Features with a small number of unique values may use less than``max_bins`` bins. In addition to the ``max_bins`` bins, one more binis always reserved for missing values. Must be no larger than 255.",255
,"categorical_features categorical_features: array-like of {bool, int, str} of shape (n_features) or shape (n_categorical_features,), default='from_dtype'Indicates the categorical features.- None : no feature will be considered categorical.- boolean array-like : boolean mask indicating categorical features.- integer array-like : integer indices indicating categorical features.- str array-like: names of categorical features (assuming the training data has feature names).- `""from_dtype""`: dataframe columns with dtype ""category"" are considered to be categorical features. The input must be an object exposing a ``__dataframe__`` method such as pandas or polars DataFrames to use this feature.For each categorical feature, there must be at most `max_bins` uniquecategories. Negative values for categorical features encoded as numericdtypes are treated as missing values. All categorical values areconverted to floating point numbers. This means that categorical valuesof 1.0 and 1 are treated as the same category.Read more in the :ref:`User Guide ` and:ref:`sphx_glr_auto_examples_ensemble_plot_gradient_boosting_categorical.py`... versionadded:: 0.24.. versionchanged:: 1.2 Added support for feature names... versionchanged:: 1.4

In [18]:
hgb_y_train_pred = hgb_model.predict(X_train_fe)
hgb_y_test_pred = hgb_model.predict(X_test_fe)

hgb_train_rmse = root_mean_squared_error(y_train, hgb_y_train_pred)
hgb_test_rmse = root_mean_squared_error(y_test, hgb_y_test_pred)

hgb_train_r2 = r2_score(y_train, hgb_y_train_pred)
hgb_test_r2 = r2_score(y_test, hgb_y_test_pred)

print(f"Train RMSE: {hgb_train_rmse:.2f}")
print(f"Test RMSE : {hgb_test_rmse:.2f}")
print(f"Train R²  : {hgb_train_r2:.3f}")
print(f"Test R²   : {hgb_test_r2:.3f}")

Train RMSE: 35390.54
Test RMSE : 45531.58
Train R²  : 0.906
Test R²   : 0.842


In [19]:
import json
from pathlib import Path

metrics_path = Path("../artifacts/gradient_boosting/metrics.json")
metrics_path.parent.mkdir(parents=True, exist_ok=True)

metrics = {}

# Base structure
metrics.setdefault("model_family", "gradient_boosting")
metrics.setdefault("models", {})

# HistGradientBoostingRegressor baseline
metrics["models"]["hist_gradient_boosting"] = {
    "holdout": {
        "train": {
            "rmse": hgb_train_rmse,
            "r2": hgb_train_r2
        },
        "test": {
            "rmse": hgb_test_rmse,
            "r2": hgb_test_r2
        }
    }
}

# Save
with open(metrics_path, "w") as f:
    json.dump(metrics, f, indent=4)


#### Saving the model

In [20]:
import joblib
path = Path("../artifacts/gradient_boosting/hgb_model.joblib")
path.parent.mkdir(parents=True, exist_ok=True)
joblib.dump(hgb_model, path)

['../artifacts/gradient_boosting/hgb_model.joblib']

### Cross Validation On Gradient Boosting Regressor

In [21]:
from sklearn.model_selection import cross_val_score

hgb_cv = HistGradientBoostingRegressor(
    max_depth=8,
    learning_rate=0.1,
    max_iter=200,
    random_state=42
)

In [22]:
# 5-fold cross-validation using RMSE
cv_scores = cross_val_score(
    hgb_cv,
    X_train_fe,
    y_train,
    cv=5,
    scoring="neg_root_mean_squared_error"
)

In [23]:
# Convert to positive RMSE
cv_rmse_scores = -cv_scores

print("CV RMSE scores:", cv_rmse_scores)
print("Mean CV RMSE :", cv_rmse_scores.mean())
print("Std CV RMSE  :", cv_rmse_scores.std())

CV RMSE scores: [45999.61206895 46475.78726165 47378.55912045 45791.50926531
 46849.82907906]
Mean CV RMSE : 46499.05935908268
Std CV RMSE  : 573.9643800860204


In [24]:
import json
from pathlib import Path
import numpy as np

metrics_path = Path("../artifacts/gradient_boosting/metrics.json")

# Load existing metrics
with open(metrics_path, "r") as f:
    metrics = json.load(f)

# Add CV results
metrics["models"]["hist_gradient_boosting"]["cross_validation"] = {
    "folds": 5,
    "rmse_scores": cv_rmse_scores.tolist(),
    "rmse_mean": float(np.mean(cv_rmse_scores)),
    "rmse_std": float(np.std(cv_rmse_scores))
}

# Save back
with open(metrics_path, "w") as f:
    json.dump(metrics, f, indent=4)

---